# Binder for custom-ner-de

## Create training labels from XML files to train custom spaCy model

###


In [ ]:
ZIP_PATH = input()
exit()

**Below cell is the logic for the parsing XML file and extracting label and converting it to spacy format.**

# todo: import functions rather than duplicating them

In [ ]:
# add words in the list to remove beforehand
word_remove = ["HÃ¤ndeklatschen"]

In [11]:
final_all_ents_tuple = []
all_sentences_present = []

#looping over all the files
for j in range(len(xml_files)):

  print("processing file=================================== ", xml_files[j])
  
  mytree = ET.parse('xml_files/'+xml_files[j])
  myroot = mytree.getroot()

  for x in myroot[1][1]:
    if x.tag.endswith('TextLine'):
      if "person" in x.attrib['custom'] or "place" in x.attrib['custom']:
 
        ents = x.attrib['custom'].split(" ")[2:]
        print(ents)
        sentence = x[-1][0].text
        all_sentences_present.append(sentence)
 
        all_ents = []
 
        for i in range(0, len(ents)):
          if ents[i] in ['person', 'place']:
            if ents[i] == 'person':
               ent = 'PERSON'
            else:
              ent = 'LOC'
 
            a = int(ents[i+1].split(":")[1][:-1])

            ## following if-else condition is written as there are some labels which has 'continued:true' means there are more word belong to current word
            if ents[i+2].endswith("}"):
              b = int(ents[i+2].split(":")[1][:-2])
            else:
              try:
                i += 4
                b1 = int(a[i+1].split(":")[1][:-1])
                if ents[i+2].endswith("}"):
                  b2 = int(ents[i+2].split(":")[1][:-2])
                else:
                  b2 = int(ents[i+2].split(":")[1][:-1])
                b = b1 + b2
              except:
                i -= 4
                b = int(ents[i+2].split(":")[1][:-1])
 
            # checking if word is part of words to remove
            if not list(set(sentence[a:a+b].split(" ")) & set(word_remove)):
              ent_tuple = [a, a+b, ent] #single tuple as per the format defined by spacy
              all_ents.append(ent_tuple)
            else:
              continue

        # following loop is written because in there are some samples which has overlapping range, this loop handles those overlapping words as they are already covered.
        all_ents_copy = all_ents.copy()
        for k in range(len(all_ents)-1):
          if all_ents[k][0] <= all_ents[k+1][0] <= all_ents[k][1] or all_ents[k][0] <= all_ents[k+1][1] <= all_ents[k][1]:
            try:
              del all_ents_copy[k+1]
            except:
              del all_ents_copy[k]

        if all_ents_copy:
          final_tuple = (sentence, {'entities' : all_ents_copy})
          print(final_tuple)
        final_all_ents_tuple.append(final_tuple)  #this variable holds all the tuples from all the files
        print("=="*50)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
['person', '{offset:0;', 'length:6;}', 'person', '{offset:8;', 'length:12;}', 'person', '{offset:22;', 'length:18;}', 'person', '{offset:42;', 'length:11;}', 'person', '{offset:55;', 'length:8;}']
('Philos, Ibn Gabirols, Jehuda ben Halewys, Ben Maimons, Spinozas,', {'entities': [[0, 6, 'PERSON'], [8, 20, 'PERSON'], [22, 40, 'PERSON'], [42, 53, 'PERSON'], [55, 63, 'PERSON']]})
['person', '{offset:0;', 'length:6;}']
('Heines kennt diese Geldprotzen nicht, die alles geringschätzen, was', {'entities': [[0, 6, 'PERSON']]})
processing file===================================  Protokoll-Zionistenkongress-Basel_1897-0022.xml
['person', '{offset:39;', 'length:14;}']
('Präsident: Das Wort hat Herr Architekt Oscar Marmorek:', {'entities': [[39, 53, 'PERSON']]})
['person', '{offset:0;', 'length:14;}']
('Oscar Marmorek: Geliebte Brüder! Ein Wundertag ist der heutigé,', {'entities': [[0, 14, 'PERSON']]})
processing file===============

In [12]:
# storing all the labels in txt file called "outfile.txt"
with open("outfile.txt", "w") as outfile:
    outfile.write("\n".join(str(item) for item in final_all_ents_tuple))

##**Part 2 :: Custom training with Spacy**

In [13]:
!pip show spacy

Name: spacy
Version: 3.3.0
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: pydantic, setuptools, langcodes, spacy-legacy, requests, numpy, cymem, typing-extensions, wasabi, pathy, spacy-loggers, tqdm, typer, catalogue, murmurhash, blis, packaging, jinja2, preshed, srsly, thinc
Required-by: fastai, en-core-web-sm


In [14]:
#More packages to install
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.util import minibatch, compounding
from spacy.training import Example
from spacy.pipeline import EntityRuler

In [15]:
!mkdir de_spacy_custom_v2

In [16]:
model = None
output_dir=Path("/content/de_spacy_custom_v2") #output folder in which trained model will be stored
n_iter=100 #number of training epochs (increase for better performance or decrease for shorter run time - rule of thumb : minimum 40 epochs required)

In [17]:
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('de')  
    print("Created blank 'de' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

Created blank 'de' model


**Add words in the list to be considered in entity ruler**

In [18]:
# list of words to be added as person for training using entity ruler
person_names = ['Gustav Gottheil', 'Max Mustermann'] #add manually names here

person_patterns = []

for i in range(len(person_names)):
  person_patterns.append({"label": "PERSON", "pattern": person_names[i]})

In [19]:
# list of words to be added as location for training using entity ruler
location_names = ['Boston', 'New-York'] #add manually places/locations/cities here

location_patterns = []

for i in range(len(location_names)):
  location_patterns.append({"label": "LOC", "pattern": location_names[i]})

In [20]:
patterns = person_patterns + location_patterns

**Creating Entity Ruler with custom patterns**

In [21]:
cfg = {"overwrite_ents": True} #add an entitiy ruler for the manual changes
nlp.add_pipe('entity_ruler', before='ner', config=cfg).add_patterns(patterns)

**Below cell is for spacy training code**

**This can take up to 3 hours to complete the training for the 100 epochs.**

In [22]:
for _, annotations in final_all_ents_tuple:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [23]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

In [24]:
optimizer = nlp.begin_training()
for itn in range(n_iter):
    random.shuffle(final_all_ents_tuple)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(final_all_ents_tuple, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)
        print("Losses", losses)

/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Familie in Rumänien weder Existenz, noch Zukunft v..." with entities "[[11, 20, 'LOC']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "die ich hauptsächsich meinem Freunde Wilhelm Gross..." with entities "[[37, 49, 'PERSON']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Losses {'ner': 33.39999669790268}
Losses {'ner': 58.9196492433548}
Losses {'ner': 86.31350994110107}
Losses {'ner': 115.95980823040009}
Losses {'ner': 143.01241368055344}
Losses {'ner': 176.18857216835022}
Losses {'ner': 200.60030257701874}
Losses {'ner': 223.89103144407272}
Losses {'ner': 252.43572023510933}
Losses {'ner': 269.8159995377064}
Losses {'ner': 296.35208162665367}
Losses {'ner': 314.64986860752106}
Losses {'ner': 330.2671166062355}
Losses {'ner': 340.58000963926315}


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "M. D. Brafmann. — Josef Schenker." with entities "[[0, 14, 'PERSON'], [17, 32, 'PERSON']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Losses {'ner': 363.5217618048191}
Losses {'ner': 373.16335804760456}
Losses {'ner': 383.20495857298374}
Losses {'ner': 393.29827906936407}
Losses {'ner': 408.0481812260696}
Losses {'ner': 415.38747267384315}
Losses {'ner': 423.5897915852838}
Losses {'ner': 445.494484708237}
Losses {'ner': 456.73170333966846}
Losses {'ner': 469.284080896643}
Losses {'ner': 477.6660954293329}
Losses {'ner': 487.25818244810216}
Losses {'ner': 498.4226133020129}
Losses {'ner': 506.661671336391}


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "feld in Brünn; für Galizien die Herren Dr. A. Salz..." with entities "[[0, 4, 'PERSON'], [8, 13, 'LOC'], [19, 26, 'LOC']...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Julius Waldmann, Oberlehrer, Georg Fischer, Moriz ..." with entities "[[0, 15, 'PERSON'], [29, 42, 'PERSON'], [43, 56, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


...

Losses {'ner': 3344.180357184301}
Losses {'ner': 3350.221008528617}
Losses {'ner': 3357.727402388169}
Losses {'ner': 3365.217263630002}
Losses {'ner': 3371.266000462354}
Losses {'ner': 3373.2493731441336}
Losses {'ner': 3376.028154823435}


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Leeds. Leeds Zionist." with entities "[[0, 5, 'LOC'], [7, 13, 'LOC']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Losses {'ner': 3386.48549694341}
Losses {'ner': 3390.0761371364997}
Losses {'ner': 3396.4629174207575}
Losses {'ner': 3402.731398514944}
Losses {'ner': 3410.343900728799}
Losses {'ner': 3417.0295359636502}


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "aufzunehmen, in PALASTINA angesiedelt zu werden." with entities "[[16, 26, 'LOC']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Goldschläger, Israel Händler, Jakob Stetter, Moses..." with entities "[[0, 12, 'PERSON'], [13, 28, 'PERSON'], [30, 43, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Losses {'ner': 6.118858558883404}
Losses {'ner': 6.142034101249671}
Losses {'ner': 6.14204498208601}
Losses {'ner': 6.142123735636129}
Losses {'ner': 6.142123767400045}
Losses {'ner': 6.502256922148063}
Losses {'ner': 6.50226835639986}
Losses {'ner': 6.502268356401817}
Losses {'ner': 6.5022683567751685}
Losses {'ner': 6.502270384644282}
Losses {'ner': 7.25330393453026}
Losses {'ner': 7.275170763013072}
Losses {'ner': 7.275170765302939}
Losses {'ner': 7.2784558529364904}
Losses {'ner': 7.28136181198173}
Losses {'ner': 7.281361812564415}
Losses {'ner': 7.281362242497459}
Losses {'ner': 9.259696788602662}
Losses {'ner': 9.259696803520885}
Losses {'ner': 9.43254049832116}
Losses {'ner': 9.432540502235405}
Losses {'ner': 9.432540504059437}
Losses {'ner': 9.432540512213956}
Losses {'ner': 9.432558688513481}
Losses {'ner': 9.432560509389507}
Losses {'ner': 9.432560510097227}
Losses {'ner': 9.432560510099764}
Losses {'ner': 9.4

In [25]:
#saving trained model in directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/de_spacy_custom_v2


In [26]:
#sample inference using trained model
for text, _ in final_all_ents_tuple[:5]:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Dr. Bernstein', 'PERSON'), ('Steiner', 'PERSON'), ('Mr. de Haas', 'PERSON'), ('London', 'LOC')]
Entities [('Isidor Schalit', 'PERSON'), ('Wien', 'LOC')]
Entities [('Herr Rubinstein', 'PERSON')]
Entities [('Palästinas', 'LOC')]
Entities [('Palästina', 'LOC')]


/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any pattern

##**Part 3 :: Inference of Custom trained model on test data**

In [27]:
# add words in this list which needs to be removed.
word_remove = ['Händeklatschen'] #seperate words with commas (,)

In [28]:
import pandas as pd #read the text file ".txt" to test the model -> rename if necessary
test_df = pd.read_csv('03_Protokoll-Zionistenkongress-Basel_1899.txt', delimiter = "\n", header=None, names=["text"])

In [29]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14740 non-null  object
dtypes: object(1)
memory usage: 115.3+ KB


In [30]:
test_df.head(30)

,text
0,Stenographisches Protokoll
1,der
2,Verhandlungen
3,des
4,ten¬
5,gresses
6,BASEL
7,15. bis 18. August 1899.
8,"WIEN, 1899."
9,"Buchdruckerei „Industrie“, Wien. — Verlag des ..."


In [31]:
all_persons = []
all_locations = []

for jj in range(len(test_df)):
  doc = nlp(test_df['text'][jj])
  persons = [ent.text for ent in doc.ents if ent.label_ == 'PERSON' and ent.text not in word_remove]
  locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC' and ent.text not in word_remove]
  all_persons.append(persons)
  all_locations.append(locations)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWarning: [W036] The component 'entity_ruler' does not have any patterns defined.
  warnings.warn(Warnings.W036.format(name=self.name))
/usr/local/lib/python3.7/dist-packages/spacy/pipeline/entityruler.py:394: UserWar

In [32]:
test_df['v2_Custom-trained_Spacy_Person'] = pd.Series(all_persons)
test_df['v2_Custom-trained_Spacy_Location'] = pd.Series(all_locations)

In [33]:
test_df.head(30)

,text,v2_Custom-trained_Spacy_Person,v2_Custom-trained_Spacy_Location
0,Stenographisches Protokoll,[Stenographisches Protokoll],[]
1,der,[],[]
2,Verhandlungen,[],[]
3,des,[],[]
4,ten¬,[],[]
5,gresses,[],[]
6,BASEL,[],[BASEL]
7,15. bis 18. August 1899.,[],[]
8,"WIEN, 1899.",[],[WIEN]
9,"Buchdruckerei „Industrie“, Wien. — Verlag des ...",[],[Wien]


In [34]:
test_df.tail(30)

,text,v2_Custom-trained_Spacy_Person,v2_Custom-trained_Spacy_Location
14710,Saffed. 17 Unterschriften.,[],[Saffed]
14711,Jaffa: 10 Unterschriften.,[],[Jaffa]
14712,Tunis.,[],[Tunis]
14713,Tunis. Ohne Unterschrift.,[Ohne Unterschrift],[Tunis]
14714,Vereinigte Staaten von Nordamerika.,[],[Vereinigte Staaten von Nordamerika]
14715,Baltimore. „Cheras Zion“.,[],[Baltimore]
14716,Dr. Schaffer.,[Dr. Schaffer],[]
14717,Boston. Solme treiter Ziont.,[Solme],[Boston]
14718,Zionistische Vereinigung.,[],[]
14719,Cincinnati. Petition mit Unterschriften.,[],[Cincinnati]


In [35]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 3 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   text                              14740 non-null  object
 1   v2_Custom-trained_Spacy_Person    14740 non-null  object
 2   v2_Custom-trained_Spacy_Location  14740 non-null  object
dtypes: object(3)
memory usage: 345.6+ KB


In [36]:
test_df.head(40)

,text,v2_Custom-trained_Spacy_Person,v2_Custom-trained_Spacy_Location
0,Stenographisches Protokoll,[Stenographisches Protokoll],[]
1,der,[],[]
2,Verhandlungen,[],[]
3,des,[],[]
4,ten¬,[],[]
5,gresses,[],[]
6,BASEL,[],[BASEL]
7,15. bis 18. August 1899.,[],[]
8,"WIEN, 1899.",[],[WIEN]
9,"Buchdruckerei „Industrie“, Wien. — Verlag des ...",[],[Wien]


In [37]:
##**These are the results of the own trained model - saves as "v2_Custom_NER_inference_results.csv" - change directory and/or name if needed**

In [38]:
test_df.to_csv('v2_Custom_NER_inference_results.csv',index=False) #saving inference results of custom trained model

##**Part 4 :: Using pre-trained German spacy Large model to detect entity**

In [39]:
!python -m spacy download de_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 567.8 MB 26 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_lg')


In [40]:
import pandas as pd

In [41]:
nlp = spacy.load("de_core_news_lg") #loading the large pre-trained spacy model for german language

In [42]:
df = pd.read_csv("v2_Custom_NER_inference_results.csv") #loads the csv of custom trained results -> change here if you renamed this file earlier

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 3 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   text                              14740 non-null  object
 1   v2_Custom-trained_Spacy_Person    14740 non-null  object
 2   v2_Custom-trained_Spacy_Location  14740 non-null  object
dtypes: object(3)
memory usage: 345.6+ KB


In [44]:
all_persons = []
all_locations = []

for jj in range(len(df)):
  doc = nlp(df['text'][jj])
  persons = [ent.text for ent in doc.ents if ent.label_ == 'PER']
  locations = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
  all_persons.append(persons)
  all_locations.append(locations)

In [45]:
df['Pre-trained_Spacy_Person'] = pd.Series(all_persons)
df['Pre-trained_Spacy_Location'] = pd.Series(all_locations)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14740 entries, 0 to 14739
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   text                              14740 non-null  object
 1   v2_Custom-trained_Spacy_Person    14740 non-null  object
 2   v2_Custom-trained_Spacy_Location  14740 non-null  object
 3   Pre-trained_Spacy_Person          14740 non-null  object
 4   Pre-trained_Spacy_Location        14740 non-null  object
dtypes: object(5)
memory usage: 575.9+ KB


In [47]:
df.to_csv("v2_Custom_NER_All_Inference_results.csv", index=False) #saving final results which has results of custom model and pre-trained spacy large model.

In [48]:
!zip -r de_spacy_custom_v2.zip de_spacy_custom_v2/

  adding: de_spacy_custom_v2/ (stored 0%)
  adding: de_spacy_custom_v2/vocab/ (stored 0%)
  adding: de_spacy_custom_v2/vocab/key2row (stored 0%)
  adding: de_spacy_custom_v2/vocab/lookups.bin (stored 0%)
  adding: de_spacy_custom_v2/vocab/vectors.cfg (stored 0%)
  adding: de_spacy_custom_v2/vocab/vectors (deflated 45%)
  adding: de_spacy_custom_v2/vocab/strings.json (deflated 73%)
  adding: de_spacy_custom_v2/meta.json (deflated 53%)
  adding: de_spacy_custom_v2/ner/ (stored 0%)
  adding: de_spacy_custom_v2/ner/model (deflated 7%)
  adding: de_spacy_custom_v2/ner/cfg (deflated 33%)
  adding: de_spacy_custom_v2/ner/moves (deflated 50%)
  adding: de_spacy_custom_v2/entity_ruler/ (stored 0%)
  adding: de_spacy_custom_v2/entity_ruler/cfg (deflated 5%)
  adding: de_spacy_custom_v2/entity_ruler/patterns.jsonl (stored 0%)
  adding: de_spacy_custom_v2/config.cfg (deflated 59%)
  adding: de_spacy_custom_v2/tokenizer (deflated 84%)


In [49]:
df.tail(50)

,text,v2_Custom-trained_Spacy_Person,v2_Custom-trained_Spacy_Location,Pre-trained_Spacy_Person,Pre-trained_Spacy_Location
14690,asui. Verein „Jerusalem.,[],['Jerusalem'],[],[]
14691,Russland.,[],['Russland'],[],[]
14692,Aus 174 Orten liefen 256 Zuschriften und Teleg...,['Telegramme'],[],[],[]
14693,personen und 213 Gruppen und Corporationen ein.,[],[],[],[]
14694,Schweiz.,[],['Schweiz'],[],[]
14695,Bern. Zionistische Studierende.,[],[],[],[]
14696,Zionistische Ortsgruppe.,[],[],[],[]
14697,Genf. Bernard Mochenson.,"['Genf.', 'Bernard Mochenson']",[],[Bernard Mochenson.],[Genf]
14698,Zionistische Ortsgruppe.,[],[],[],[]
14699,Locle. Dr. Beredka.,['Dr. Beredka'],['Locle'],[Locle.],[]


##**Part 5 :: Calculating Accuracy score of Custom model and Spacy large model**

In order to run below code please make sure below files are in same folder as notebook, if not, change path accordingly.

- This Notebook
- extracted_entities.txt
- custom trained model folder

In [ ]:
!ls -lh

total 188K
drwxr-xr-x 5 root root 4.0K May 27 12:44 de_spacy_custom_v2
-rw-r--r-- 1 root root 179K May 27 12:43 extracted_entities.txt
drwxr-xr-x 1 root root 4.0K May 17 13:39 sample_data


In [ ]:
import warnings
warnings.filterwarnings('ignore')
from spacy.training import Example
import spacy

In [ ]:
def load_data():
    """
    function to load entity data

    input ::
        
    output ::
        - Entity data to use for accuracy calculation
    """

    print("loading data...")
    file1=open('extracted_entities.txt')

    lines = file1.readlines()

    for i in range(len(lines)):
        lines[i] = eval(lines[i])

    return lines

In [ ]:
def load_custom_spacy_model(model_path):
    """
    function to load the custom trained spacy model

    input ::
        - folder_path : folder which contains model

    output ::
        - model
    """

    print("Loading model from {0}\n".format(model_path))
    nlp = spacy.load(model_path)

    return nlp

In [ ]:
def calculate_custom_model_accuracy(data):
    """
    function to calculate accuracy of custom trained entity model

    input ::
        - list containing entity data
        
    output ::
        - accuracy metrics 
    """

    nlp = load_custom_spacy_model("de_spacy_custom_v2")

    print("Calculating score...")
    new_test_data = []

    for text, annots in data:
        new_test_data.append(Example.from_dict(nlp.make_doc(text), annots))

    scores_model = nlp.evaluate(new_test_data)

    #print scores that you want
    precision_model = scores_model["ents_p"]
    recall_model = scores_model["ents_r"]
    f_score_model = scores_model["ents_f"]
    scores_entities = scores_model["ents_per_type"]

    print("================ Accuracy scores using custom trained model =================\n")
   
    print("================= Overall scores =================\n")
    print("Precision : ",precision_model)
    print("Recall : ",recall_model)
    print("F1 Score : ",f_score_model)
   
    print("\n================= Entity wise score =================\n")
   
    print("============= Person Entity score =================\n")
    print("Precision : ",scores_entities['PERSON']['p'])
    print("Recall : ",scores_entities['PERSON']['r'])
    print("F1 Score : ",scores_entities['PERSON']['r'])

    print("\n============= Location Entity score =================\n")
    print("Precision : ",scores_entities['LOC']['p'])
    print("Recall : ",scores_entities['LOC']['r'])
    print("F1 Score : ",scores_entities['LOC']['r'])


def calculate_pre_trained_model_score(data):
    """
    function to calculate accuracy of custom trained entity model

    input ::
        - list containing entity data
        
    output ::
        - accuracy metrics 
    """

    # using spact large german model
    nlp = spacy.load("de_core_news_lg")

    print("\n\nCalculating score...")
    new_test_data = []

    for text, annots in data:
        new_test_data.append(Example.from_dict(nlp.make_doc(text), annots))

    scores_model = nlp.evaluate(new_test_data)

    #print scores that you want
    precision_model = scores_model["ents_p"]
    recall_model = scores_model["ents_r"]
    f_score_model = scores_model["ents_f"]
    scores_entities = scores_model["ents_per_type"]

    print("\n================ Accuracy scores using Pre-trained large model =================\n")
   
    print("================= Overall scores =================\n")
    print("Precision : ",precision_model)
    print("Recall : ",recall_model)
    print("F1 Score : ",f_score_model)
   
    print("\n================= Entity wise score =================\n")
   
    print("============= Person Entity score =================\n")
    print("Precision : ",scores_entities['PERSON']['p'])
    print("Recall : ",scores_entities['PERSON']['r'])
    print("F1 Score : ",scores_entities['PERSON']['r'])

    print("\n============= Location Entity score =================\n")
    print("Precision : ",scores_entities['LOC']['p'])
    print("Recall : ",scores_entities['LOC']['r'])
    print("F1 Score : ",scores_entities['LOC']['r'])

In [ ]:
#loading data
data = load_data()

loading data...


In [ ]:
#accuracy score of custom trained model
calculate_custom_model_accuracy(data)

Loading model from de_spacy_custom_v2

Calculating score...
================ Accuracy scores using custom trained model =================

================= Overall scores =================

Precision :  0.9545762711864407
Recall :  0.9968141592920354
F1 Score :  0.9752380952380952

================= Entity wise score =================

============= Person Entity score =================

Precision :  0.9260543038705951
Recall :  0.9956521739130435
F1 Score :  0.9956521739130435

============= Location Entity score =================

Precision :  0.9950779327317474
Recall :  0.9983539094650206
F1 Score :  0.9983539094650206


In [ ]:
#accuracy score of spacy large model
calculate_pre_trained_model_score(data)



Calculating score...

================ Accuracy scores using Pre-trained large model =================

================= Overall scores =================

Precision :  0.21793635486981677
Recall :  0.16
F1 Score :  0.18452745458256786

================= Entity wise score =================

============= Person Entity score =================

Precision :  0.0
Recall :  0.0
F1 Score :  0.0

============= Location Entity score =================

Precision :  0.6943164362519201
Recall :  0.3720164609053498
F1 Score :  0.3720164609053498
